##Setup

In [9]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install -U transformers
#!pip install vllm==0.5.3.post1
!pip install -U torch
!pip install pandas

In [11]:
!pip install datasets

In [17]:
from datasets import load_dataset
judgebench = load_dataset("ScalerLab/JudgeBench", split="claude")

In [18]:
judgebench[0]

{'pair_id': 'b5ce1305-50fe-5a5e-b785-325ab15c6d2b',
 'original_id': 6603,
 'source': 'mmlu-pro-health',
 'question': "In the brain stem, pathways for:\n(A) discriminative touch decussate in the pyramids.\n(B) proprioception decussate in the medial lemniscus\n(C) skilled movements decussate in the corticospinal tract\n(D) proprioception decussate in the pyramids\n(E) skilled movements decussate in the medial lemniscus\n(F) skilled motor movements decussate in the pyramids.\n(G) nociception decussate in the medial lemniscus\n(H) nociception decussate in the pyramids\n(I) skilled motor movements decussate in the medial lemniscus\n(J) discriminative touch decussate in the medial lemniscus\nIf you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.\nLet's think step by step.",
 'response_model': 'claude-3-5-sonnet-20240620',
 'respo

In [19]:
def get_vanilla(question, response_a, response_b):
  return f'''
  You are a helpful assistant in evaluating the quality of the outputs for a given instruction. Your goal is to select the best output for the given instruction.
  Select the Output (a) or Output (b) that is better for the given instruction. The two outputs are generated by two different AI chatbots respectively.
  Do NOT provide any explanation for your choice.
  Do NOT say both / neither are good.
  You should answer using ONLY "Output (a)" or "Output (b)". Do NOT output any other words.
  # Instruction: {question}
  # Output (a): {response_a}
  # Output (b): {response_b}
  # Which is better, Output (a) or Output (b)? Your response should be either "Output (a)" or "Output (b)":
  '''

In [20]:
def get_arena(prompt, answer_a, answer_b):
  return f'''
    Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user prompt displayed below. You will be given assistant A's answer and assistant B's answer. Your job is to evaluate which assistant's answer is better.

    Begin your evaluation by generating your own answer to the prompt. You must provide your answers before judging any answers.

    When evaluating the assistants' answers, compare both assistants' answers with your answer. You must identify and correct any mistakes or inaccurate information.

    Then consider if the assistant's answers are helpful, relevant, and concise. Helpful means the answer correctly responds to the prompt or follows the instructions. Note when user prompt has any ambiguity or more than one interpretation, it is more helpful and appropriate to ask for clarifications or more information from the user than providing an answer based on assumptions. Relevant means all parts of the response closely connect or are appropriate to what is being asked. Concise means the response is clear and not verbose or excessive.

    Then consider the creativity and novelty of the assistant's answers when needed. Finally, identify any missing important information in the assistants' answers that would be beneficial to include when responding to the user prompt.

    After providing your explanation, you must output only one of the following choices as your final verdict with a label:

    1. Assistant A is better: [[A>B]]
    2. Assistant B is better: [[B>A]]

    Example output: "My final verdict is Assistant A is better: [[A>B]]".

    |User Prompt|>
    {prompt}

    <|The Start of Assistant A's Answer|>
    {answer_a}
    <|The End of Assistant A's Answer|>

    <|The Start of Assistant B's Answer|>
    {answer_b}
    <|The End of Assistant B's Answer|>

    <|Your Answer|>
    '''

In [24]:
from datasets import Dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json

def preprocess(data):
  batch = {}
  batch['text'] = [get_vanilla(data['question'][i], data['response_A'][i], data['response_B'][i]) for i in range(len(data['question']))]
  batch['id'] = data['pair_id'] # Access the 'pair_id' column directly
  return batch

def compute_metric():
    return

def collate(data):
    return {'text': [ex['text'] for ex in data],
            'id': [ex['id'] for ex in data],
            }

batch_size = 8
print('loading dataset...\n')
# train_data = Dataset.from_pandas(judgebench)
train_data = judgebench
# train_data = train_data.select_columns(['full_text', 'text_id_kaggle'])
train_data = train_data.map(preprocess, batched=True, batch_size=batch_size)
train_dataloader = DataLoader(train_data,
                               batch_size=batch_size, shuffle=False,
                               collate_fn=collate)

loading dataset...



Map:   0%|          | 0/270 [00:00<?, ? examples/s]

##LLM grading

In [46]:
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch import cuda, tensor, no_grad
import torch
from tqdm.auto import tqdm
import json

device = 'cuda' if cuda.is_available() else 'cpu'
pipe = pipeline(
    "text-generation",
    model="google/gemma-1.1-2b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    use_cache=False,
)

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cuda:0


In [47]:
def extract_pairwise_result(raw_output):
    # print("raw:", raw_output[-20:])
    output = raw_output[-20:]
    if "Output (a)" in output: # Check if "Output (a)" is present in the raw output
        return "A"
    elif "Output (b)" in output: # Check if "Output (b)" is present in the raw output
        return "B"
    else:
        return output

In [48]:
import csv
from tqdm import tqdm
from torch import no_grad

epochs = 1

num_training_steps = epochs * len(train_dataloader)
progress_bar = tqdm(range(num_training_steps))

with open('/content/drive/MyDrive/senior_thesis/part_3/vanilla_outputs/gemma-1.1-2b-it.csv', 'w', newline='') as out_file:
    csv_writer = csv.writer(out_file, quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['id', 'pred'])  # Write header

    with no_grad():
        for _ in range(epochs):
            for _, batch in enumerate(train_dataloader):
                id = batch['id']
                out = pipe(batch['text'], max_new_tokens=216, pad_token_id=pipe.tokenizer.eos_token_id)
                for i in range(len(batch['text'])):
                    generated_text = out[i][0]["generated_text"]
                    model_response = generated_text.split("<start_of_turn>model\n")[-1].strip()
                    content = extract_pairwise_result(model_response)
                    csv_writer.writerow([id[i], content])  # CSV writer handles quoting automatically
                progress_bar.update()




100%|██████████| 34/34 [18:26<00:00, 32.54s/it]



  3%|▎         | 1/34 [00:04<02:19,  4.23s/it]


  6%|▌         | 2/34 [00:08<02:23,  4.48s/it]


  9%|▉         | 3/34 [00:13<02:15,  4.38s/it]


 12%|█▏        | 4/34 [00:25<03:49,  7.66s/it]


 15%|█▍        | 5/34 [00:30<03:10,  6.59s/it]


 18%|█▊        | 6/34 [00:35<02:45,  5.92s/it]


 21%|██        | 7/34 [00:43<02:59,  6.65s/it]


 24%|██▎       | 8/34 [00:50<02:57,  6.83s/it]


 26%|██▋       | 9/34 [00:52<02:10,  5.21s/it]


 29%|██▉       | 10/34 [00:57<02:05,  5.24s/it]


 32%|███▏      | 11/34 [01:01<01:50,  4.82s/it]


 35%|███▌      | 12/34 [01:08<01:59,  5.45s/it]


 38%|███▊      | 13/34 [01:15<02:05,  5.99s/it]


 41%|████      | 14/34 [01:24<02:19,  6.99s/it]


 44%|████▍     | 15/34 [01:29<02:01,  6.39s/it]


 47%|████▋     | 16/34 [01:33<01:41,  5.62s/it]


 50%|█████     | 17/34 [01:39<01:36,  5.70s/it]


 53%|█████▎    | 18/34 [01:43<01:25,  5.33s/it]


 56%|█████▌    | 19/34 [01:48<01:14,  4.99s/it]


 59%|